In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
# Load data
data = pd.read_csv(".csv")

# Separate features and target variable
X = data.drop('RiskLevel', axis=1)
y = data['RiskLevel']

# Map 'RiskLevel' labels to numerical values
label_mapping = {'low risk': 0, 'mid risk': 1, 'high risk': 2}
y = y.map(label_mapping)

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)
print(y)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# XGBoost model
xgb_model = xgb.XGBClassifier()

# TensorFlow model
tf_model = Sequential()
tf_model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
tf_model.add(Dense(128, activation='relu'))
tf_model.add(Dense(128, activation='relu'))
tf_model.add(Dense(3, activation='softmax'))

# Compile TensorFlow model
tf_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# Train XGBoost model
xgb_model.fit(X_train, y_train)

# Train TensorFlow model
tf_model.fit(X_train, y_train, epochs=150, batch_size=32)


In [ ]:
# Generate XGBoost predictions
xgb_preds = xgb_model.predict(X_val)

# Generate TensorFlow predictions
tf_probs = tf_model.predict(X_val)
tf_preds = np.round(tf_probs).astype(int)


In [ ]:
# Concatenate base model predictions with the original features
meta_X = np.concatenate((X_val, xgb_preds.reshape(-1, 1), tf_preds), axis=1)

# Build XGBoost meta model
meta_model = xgb.XGBClassifier()

# Define hyperparameters for grid search
params = {'max_depth': [3, 5, 7], 'learning_rate': [0.1, 0.01], 'n_estimators': [100, 200, 300]}

# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(meta_model, params, cv=5, scoring='accuracy')
grid_search.fit(meta_X, y_val)

# Get the best meta model
best_meta_model = grid_search.best_estimator_


In [ ]:
# Generate predictions from the meta model
meta_preds = best_meta_model.predict(meta_X)

# Evaluate the meta model
print(classification_report(y_val, meta_preds))


In [ ]:
# Convert TensorFlow model to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

# Save the TensorFlow model to H5 format
tf_model.save('model.h5')
